<a href="https://colab.research.google.com/github/blacktalenthubs/Dynamic-Application-Security-Testing/blob/main/week5_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 comprehensive setup guide for configuring a Security Information and Event Management (SIEM) solution using the ELK Stack (Elasticsearch, Logstash, and Kibana) along with Filebeat to collect and send log data for an incident response .

---

# Incident Response SIEM Tools Setup Guide

## Part 1: Prerequisites

### Step 1: Install Docker and Docker Compose
1. **Install Docker**:
   ```bash
   brew install --cask docker
   ```
   - Start Docker Desktop from your Applications folder.

2. **Install Docker Compose**:
   ```bash
   brew install docker-compose
   ```

## Part 2: Set Up the ELK Stack with Docker

### Step 1: Create a Directory for Docker Setup
```bash
mkdir elk-docker
cd elk-docker
```

### Step 2: Create `docker-compose.yml` File
```bash
nano docker-compose.yml
```

### Step 3: Add the Following Configuration to `docker-compose.yml`
```yaml
version: '3'
services:
  elasticsearch:
    image: docker.elastic.co/elasticsearch/elasticsearch:7.9.3
    container_name: elasticsearch
    environment:
      - discovery.type=single-node
      - ES_JAVA_OPTS=-Xms512m -Xmx512m
      - network.host=0.0.0.0
    ports:
      - "9200:9200"
      - "9300:9300"
    volumes:
      - esdata:/usr/share/elasticsearch/data
    platform: linux/amd64

  logstash:
    image: docker.elastic.co/logstash/logstash:7.9.3
    container_name: logstash
    volumes:
      - ./logstash-pipeline:/usr/share/logstash/pipeline
    ports:
      - "5044:5044"
    depends_on:
      - elasticsearch
    platform: linux/amd64

  kibana:
    image: docker.elastic.co/kibana/kibana:7.9.3
    container_name: kibana
    environment:
      - ELASTICSEARCH_HOSTS=http://elasticsearch:9200
      - server.host=0.0.0.0
    ports:
      - "5601:5601"
    depends_on:
      - elasticsearch
    platform: linux/amd64

volumes:
  esdata:
    driver: local
```

### Step 4: Create Logstash Pipeline Configuration
1. **Create a Directory for Logstash Pipeline Configuration**:
   ```bash
   mkdir logstash-pipeline
   ```

2. **Create a `logstash.conf` File in the `logstash-pipeline` Directory**:
   ```bash
   nano logstash-pipeline/logstash.conf
   ```

3. **Add the Following Configuration to `logstash.conf`**:
   ```ruby
   input {
     beats {
       port => 5044
     }
   }

   filter {
     grok {
       match => { "message" => "%{COMMONAPACHELOG}" }
     }
   }

   output {
     elasticsearch {
       hosts => ["http://elasticsearch:9200"]
       index => "logs-%{+YYYY.MM.dd}"
     }
   }
   ```

### Step 5: Start the ELK Stack
```bash
docker-compose up -d
```

### Step 6: Verify the Setup
1. **Check the Status of the Containers**:
   ```bash
   docker-compose ps
   ```

2. **Access Kibana Web Interface**:
   - Open your web browser and navigate to `http://localhost:5601`.

## Part 3: Set Up Filebeat for Data Collection

### Step 1: Install Homebrew (if not already installed)
```bash
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
```

### Step 2: Install Filebeat using Homebrew
```bash
brew tap elastic/tap
brew install elastic/tap/filebeat-full
```

### Step 3: Configure Filebeat
1. **Open the Filebeat Configuration File**:
   ```bash
   sudo nano /usr/local/etc/filebeat/filebeat.yml
   ```

2. **Add the Following Configuration**:
   ```yaml
   filebeat.inputs:
   - type: log
     enabled: true
     paths:
       - /var/log/*.log

   output.elasticsearch:
     hosts: ["http://localhost:9200"]
     username: "elastic"
     password: "<your_elasticsearch_password>"

   setup.kibana:
     host: "http://localhost:5601"
   ```

### Step 4: Enable and Start Filebeat
```bash
sudo filebeat modules enable system
sudo filebeat setup
sudo brew services start filebeat-full
```

## Part 4: Simulate Security Incidents

### Step 1: Simulate Brute Force Attack
- Use a tool like Hydra to simulate brute force attacks on an SSH server within the host machine.
- Example command using Hydra:
  ```bash
  hydra -l root -P /usr/share/wordlists/rockyou.txt ssh://localhost
  ```

### Step 2: Simulate Malware Infection
- Create a script or use a tool to generate logs that mimic malware activity, such as unauthorized file access or process creation.
- Example: Create a script that generates fake logs of suspicious file access.

### Step 3: Simulate DDoS Attack
- Use a tool like LOIC (Low Orbit Ion Cannon) to simulate a DDoS attack on a web server.
- Ensure that the attack is within a controlled environment to avoid legal issues.

## Part 5: Collecting and Analyzing Logs in Kibana

### Step 1: Verify Logs in Elasticsearch
- Check if the logs are being collected in Elasticsearch:
  ```bash
  curl -X GET "http://localhost:9200/_cat/indices?v"
  ```

### Step 2: Create Index Patterns in Kibana
- Open Kibana at `http://localhost:5601`.
- Navigate to `Management` > `Index Patterns`.
- Create a new index pattern (e.g., `filebeat-*`).

### Step 3: Visualize Data in Kibana
- Navigate to `Discover` to explore your log data.
- Create visualizations and dashboards:
  - Navigate to `Visualize` to create visualizations.
  - Use visualizations like histograms, pie charts, and maps to represent your data.
  - Navigate to `Dashboard` to combine visualizations into comprehensive dashboards.

## Part 6: Teaching Incident Response

### Step 1: Incident Detection
- Show how to detect anomalies using Kibana visualizations.
- Use dashboards to monitor real-time data and identify potential security incidents.

### Step 2: Incident Analysis
- Demonstrate how to drill down into specific logs to analyze incidents.
- Use filters and search queries in Kibana to isolate relevant log entries.

### Step 3: Incident Response
- Teach how to respond to incidents based on the analysis.
- Discuss steps like isolating affected systems, mitigating attacks, and collecting forensic evidence.

### Step 4: Reporting and Documentation
- Show how to generate reports from Kibana dashboards.
- Discuss the importance of documenting incidents and response actions.

## Example Scenario: Brute Force Attack Detection

### Step 1: Simulate Brute Force Attack
- Use Hydra to simulate an SSH brute force attack as shown above.

### Step 2: Collect Logs
- Ensure Filebeat is collecting SSH logs (usually located in `/var/log/auth.log`).

### Step 3: Create Visualization in Kibana
- In Kibana, create a line chart to show failed login attempts over time.
- Use the following steps:
  - Navigate to `Visualize` > `Create Visualization` > `Line`.
  - Select the `filebeat-*` index pattern.
  - Configure the X-axis with a date histogram of the timestamp.
  - Configure the Y-axis with a count of failed login attempts.

### Step 4: Analyze and Respond
- Show the spike in failed login attempts on the Kibana dashboard.
- Discuss response strategies, such as blocking the attacking IP, enhancing password policies, and setting up alerts for similar patterns.

## Conclusion

By following this setup guide, you will create a practical, hands-on environment for teaching incident response using the ELK Stack. Students will gain experience in detecting, analyzing, and responding to security incidents, which is essential for cybersecurity professionals.